## **First Question:**

The research will focus on the following question:

**"Can sentiment analysis technology be used in a realistic business environment to understand customer feedback left on social media?"**

Then we will analyze some textual reviews and we will label them with our rating model, and we will understand how much the rating of the classified reviews differs from the automatic system, from the rating
of real reviews (assuming that the stars left by users are balanced
with the textual review left).


In [ ]:
#importing libraries
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
import sys
import numpy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [ ]:
#adding label with the three categories
import pandas as pd
import numpy as np

df = pd.read_csv('Datafiniti_Hotel_Reviewscomplete.csv')

min = 0
max = 5
col = 'reviews.rating'
conditions = [(df[col] >= min) & (df[col] < min+3),
              (df[col] == min + 3) & (df[col] == min+3),
              (df[col] > min+3)  & (df[col] <= max)]
choices = ["Bad","Neutral","Good"]
df["Evaluation"] = np.select(conditions, choices, default=np.nan)

df.to_csv('Datafiniti_Hotel_Reviewscomplete.csv', index=False)

FileNotFoundError: ignored

In [ ]:
#SPLIT & BALANCE
#split train and test and do the balance on the train
#traib 75% test 25%
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import seaborn as sns
from csv import reader
import matplotlib.pyplot as plt


#split the data into train and test set
#reading dataset
readCsv = pd.read_csv("Datafiniti_Hotel_Reviewscomplete.csv")


train,test = train_test_split(readCsv, test_size=0.25, random_state=1)
#save the data
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

#PLOT 1
plt.title("Class of imbalanced training Set")
# read a tips.csv file from seaborn library
df = pd.read_csv("train.csv")
# count plot along x axis
sns.countplot(x="Evaluation", data = df , palette="magma")
# Show the plot
plt.show()


#do rebalncing training set
class_0 = train[train['Evaluation'] == "Good"]
class_1 = train[train['Evaluation'] == "Bad"]
class_2 = train[train['Evaluation'] == "Neutral"]
class_count_0, class_count_1, class_count_2 = train['Evaluation'].value_counts()
class_0_under = class_0.sample(class_count_1)
test_under = pd.concat([class_0_under, class_1, class_2], axis=0)


test_under.to_csv("BalancedTrain.csv", index=None)
BalancedTrain = pd.read_csv('BalancedTrain.csv')

#PLOT2
plt.title("Class of balanced training Set")
# read a tips.csv file from seaborn library
df = pd.read_csv("BalancedTrain.csv")
# count plot along x axis
sns.countplot(x="Evaluation", data = df , palette="magma")
# Show the plot
plt.show()

In [ ]:
#TEXT RETRIVE & ELABORATION TEST & TRAIN
#TEXT RETRIEVE and CLEANING
#1. lower case
#2. remove punctuation
#3. remove mention @...
#4 remove link http:....
#5. remove emoji

#TEXT ELABORATION
#1. tokenization
#2. stop word filtering
#3. lematizzation




#reading the data
test_csv = pd.read_csv('test.csv') 
train_csv = pd.read_csv('BalancedTrain.csv')


train_X_non = train_csv['reviews.text'] #without preprocessing
train_y = train_csv['Evaluation']   #only evaluation
test_X_non = test_csv['reviews.text'] 
test_y = test_csv['Evaluation']  
train_X_cleaned=[]
test_X_cleaned=[]
train_X=[] #with preprocessing
test_X=[]


emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#Cleaning Train
import re
for i in range(0, len(train_X_non)):
    review = train_X_non[i]
    review = str(review).lower()
    review = review.translate(str.maketrans('', '', string.punctuation)) #remove puntuaction
    review = re.sub('@[^\s]+','',review) #remove mention
    review = re.sub('http[^\s]+','',review) #remove link
    review=emoji_pattern.sub(r'', review) # remove emoji
    train_X_cleaned.append(review)

#Cleaning Test
import re
for i in range(0, len(test_X_non)):
    review = test_X_non[i]
    review = str(review).lower()
    review = review.translate(str.maketrans('', '', string.punctuation))
    review = re.sub('@[^\s]+','',review) #remove mention
    review = re.sub('http[^\s]+','',review) #remove link
    review=emoji_pattern.sub(r'', review) # remove emoji
    test_X_cleaned.append(review)

lemmatizer = WordNetLemmatizer()
#Preprocessing Train
stop_words=set(stopwords.words("english"))
ps = PorterStemmer()
filtered_sent=[]
for i in range(0, len(train_X_cleaned)):
    review = train_X_cleaned[i]
    review=word_tokenize(review) #tokenization
    for w in review:             #stop word filtering
        if w not in stop_words:
            filtered_sent.append(w)
    review=filtered_sent
    filtered_sent=[]
    for w in review:            #stemming
        filtered_sent.append(lemmatizer.lemmatize(w))
    review=filtered_sent
    filtered_sent=[]
    review = ' '.join(review)
    train_X.append(review)
    
#Preprocessing Test
stop_words=set(stopwords.words("english"))
ps = PorterStemmer()
filtered_sent=[]
for i in range(0, len(test_X_cleaned)):
    review = test_X_cleaned[i]
    review=word_tokenize(review) #tokenization
    for w in review:             #stop word filtering
        if w not in stop_words:
            filtered_sent.append(w)
    review=filtered_sent
    filtered_sent=[]
    for w in review:            #stemming
        filtered_sent.append(lemmatizer.lemmatize(w))
    review=filtered_sent
    filtered_sent=[]
    review = ' '.join(review) #CON QUESTA UNISCO
    test_X.append(review)
    

In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer( max_features=5000,lowercase=True, analyzer='word', stop_words= 'english',ngram_range=(1,1))
X_train_counts=count_vect.fit_transform(raw_documents=train_X)

tfidf_transformer=TfidfTransformer()
X_train_tf=tfidf_transformer.fit_transform(X_train_counts)

X_test_counts=count_vect.fit_transform(raw_documents=test_X)
X_test_tf=tfidf_transformer.fit_transform(X_test_counts)


feature_names = count_vect.get_feature_names()
dense = X_train_tf.todense()
denselist = dense.tolist()
df2 = pd.DataFrame(denselist, columns=feature_names)
print(df2)

In [ ]:
#serve a trasformare le label in numeri
import copy
train_y_clf = copy.deepcopy(train_y)
test_y_clf = copy.deepcopy(test_y)

df_train_y=train_y
for i  in range(0, len(df_train_y)):
    if df_train_y[i]=="Good":
        df_train_y[i]=2
    elif df_train_y[i]=="Neutral":
        df_train_y[i]=1
    else :
        df_train_y[i]=-1

df_test_y = test_y
for i  in range(0, len(df_test_y)):
    if df_test_y[i]=="Good":
        df_test_y[i]=2
    elif df_test_y[i]=="Neutral":
        df_test_y[i]=1
    else :
        df_test_y[i]=-1
        
df_train_y.to_csv('LabelToNumber-train.csv', index=None)
df_test_y.to_csv('LabelToNumber-test.csv')

df_train_y = pd.read_csv('LabelToNumber-train.csv')
df_test_y = pd.read_csv('LabelToNumber-test.csv')

In [ ]:
#KNN 
knn = KNeighborsClassifier(n_neighbors=200,weights='uniform',algorithm='auto', n_jobs=-1)
train_sizes, train_scores, valid_scores = learning_curve(knn, X_train_tf, np.ravel(df_train_y.astype(int)),cv=10)
y_pred = cross_val_predict(knn,X_train_tf,  np.ravel(df_train_y.astype(int)), cv=10)
results = cross_validate(knn,X_train_tf, np.ravel(df_train_y.astype(int)),cv=10,scoring='neg_mean_squared_error')

accuracy = cross_val_score(knn,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(knn,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(knn,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(knn,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 


print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')



#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a KNN model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
#MULTINOMIAL DB
naive_bayes_classifier = MultinomialNB(alpha=0.5)
train_sizes, train_scores, valid_scores = learning_curve(naive_bayes_classifier, X_train_tf, np.ravel(df_train_y.astype(int)),cv=10)
y_pred = cross_val_predict(naive_bayes_classifier,X_train_tf,  np.ravel(df_train_y.astype(int)), cv=10)
results = cross_validate(naive_bayes_classifier,X_train_tf, np.ravel(df_train_y.astype(int)),cv=10,scoring='neg_mean_squared_error')

accuracy = cross_val_score(naive_bayes_classifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(naive_bayes_classifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(naive_bayes_classifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(naive_bayes_classifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 

print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')
#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a Multinomial NB model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
# BAGGING
bgclassifier = BaggingClassifier(base_estimator=None, n_estimators=30, bootstrap_features=False, oob_score=False, n_jobs=-1)

accuracy = cross_val_score(bgclassifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(bgclassifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(bgclassifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(bgclassifier,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 

print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')
#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a BAGGING model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
#DECISION TREE
decison_gini = DecisionTreeClassifier(max_depth=12,criterion='entropy')
train_sizes, train_scores, valid_scores = learning_curve(decison_gini, X_train_tf, np.ravel(df_train_y.astype(int)),cv=10)
y_pred = cross_val_predict(decison_gini,X_train_tf,  np.ravel(df_train_y.astype(int)), cv=10)
results = cross_validate(decison_gini,X_train_tf, np.ravel(df_train_y.astype(int)),cv=10,scoring='neg_mean_squared_error')

accuracy = cross_val_score(decison_gini,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(decison_gini,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(decison_gini,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(decison_gini,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 

print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')
#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a DECISION TREE model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
#ADA BOOST
ada = AdaBoostClassifier(n_estimators=150,learning_rate=1)
train_sizes, train_scores, valid_scores = learning_curve(ada, X_train_tf, np.ravel(df_train_y.astype(int)),cv=10)
y_pred = cross_val_predict(ada,X_train_tf,  np.ravel(df_train_y.astype(int)), cv=10)
results = cross_validate(ada,X_train_tf, np.ravel(df_train_y.astype(int)),cv=10,scoring='neg_mean_squared_error')

accuracy = cross_val_score(ada,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(ada,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(ada,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(ada,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 

print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')
#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a ADA BOOST model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
#SVM
clf = svm.SVC(kernel='poly' ,degree=3,cache_size=500)
train_sizes, train_scores, valid_scores = learning_curve(clf, X_train_tf, np.ravel(df_train_y.astype(int)),cv=10)
y_pred = cross_val_predict(clf,X_train_tf,  np.ravel(df_train_y.astype(int)), cv=10)
results = cross_validate(clf,X_train_tf, np.ravel(df_train_y.astype(int)),cv=10,scoring='neg_mean_squared_error')

accuracy = cross_val_score(clf,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(clf,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(clf,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(clf,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 

print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')
#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a SVM model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
#RANDOM FOREST
forest = RandomForestClassifier(n_estimators=60,max_depth=8, criterion='gini',max_features='auto',bootstrap=False, n_jobs=-1)
train_sizes, train_scores, valid_scores = learning_curve(forest, X_train_tf, np.ravel(df_train_y.astype(int)),cv=10)
y_pred = cross_val_predict(forest,X_train_tf,  np.ravel(df_train_y.astype(int)), cv=10)
results = cross_validate(forest,X_train_tf, np.ravel(df_train_y.astype(int)),cv=10,scoring='neg_mean_squared_error')

accuracy = cross_val_score(forest,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='accuracy') 
precision = cross_val_score(forest,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='precision_macro')
recall = cross_val_score(forest,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='recall_macro') 
f1 = cross_val_score(forest,X_train_tf,np.ravel(df_train_y.astype(int)), cv=10, scoring='f1_macro') 


print("\nAccuracy: ", accuracy) 
acc_round = numpy.round(accuracy.mean(),2)
print("\nAccuracy mean: ", acc_round)
print("\nPrecision: ", precision)
prec_round = numpy.round(precision.mean(),2)
print("\nPrecision mean: ", prec_round)
print("\nRecall: ", recall)
rec_round = numpy.round(recall.mean(),2)
print("\nRecall mean: ", rec_round)
print("\nF1: ", f1)
f1_round = numpy.round(f1.mean(),2)
print("\nF1 mean: ", f1_round)


print('Training scores:\n\n', train_scores)
print('\n', '-' * 70) # separator to make the output easy to read
print('\nValidation scores:\n\n', valid_scores)

train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = valid_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))
print('\n\n\n')
#print (resultstot)
plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training accuracy')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation accuracy')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for a RANDOM FOREST model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1.25)
plt.show()

In [ ]:
#T TEST FOR THE TWO BEST CLASSIFIER
from mlxtend.evaluate import paired_ttest_kfold_cv

t, p = paired_ttest_kfold_cv(estimator1=clf,
                             estimator2=bgclassifier,
                             X=X_train_tf,
                             y=np.ravel(df_train_y.astype(int)), 
                             random_seed=1)

print('p value: %.6f' % p)

In [ ]:
#PLOT THE DIFFERENCE BETWEEN N-GRAM FOR THE BEST CLASSIFICATOR 
i=1
acc = 'n-grams = '
df = pd.DataFrame()
while i<=5:
    #TF-IDF
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.feature_extraction.text import CountVectorizer
    tf_idf = TfidfVectorizer(max_features=5000, lowercase=True, analyzer='word', stop_words= 'english',ngram_range=(1,i))
    #applying tf idf to training data
    X_train_tf = tf_idf.fit_transform(train_X)   #perche fit e trasform sul train e non sul test
    #applying tf idf to training data
    X_train_tf = tf_idf.transform(train_X)
    #transforming test data into tf-idf matrix
    X_test_tf = tf_idf.transform(test_X)
    
    names =["Bagging + TFIDF"]
    bgclassifier = BaggingClassifier(base_estimator=None, n_estimators=30, bootstrap_features=False, oob_score=False, n_jobs=-1)

    classifiers = [
        bgclassifier
    ]
    #iterate on classifiers
    scores=[]
    for name, clfs in zip(names,classifiers):
        clfs.fit(X_train_tf, train_y_clf)
        score= clfs.score(X_test_tf,test_y_clf)
        scores.append(score)


   
    df['Classifier_Name'] = names
    newColumn=acc +str(i)
    df[newColumn] = scores
    i=i+1


    #sns.set(style="whitegrid")
    #ax = sns.barplot(x="Accuracy score", y="Classifier_Name", data=df)
    #plt.show()

#print(df)
sns.color_palette("mako")
dfm = df.melt('Classifier_Name', var_name='cols', value_name='vals')
#print(dfm)
g = sns.catplot(x="cols", y="vals", hue='Classifier_Name', data=dfm, kind='point' , palette=sns.color_palette(['green', 'blue','red','yellow','purple','black']))
g.fig.set_size_inches(15,5)
plt.show()


In [ ]:
#optimized TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
tf_idf = TfidfVectorizer(max_features=5000, lowercase=True, analyzer='word', stop_words= 'english',ngram_range=(1,2))
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)  #perche fit e trasform sul train e non sul test
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

feature_names = tf_idf.get_feature_names()
dense = X_train_tf.todense()
denselist = dense.tolist()
df2 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
#bagging CLASSIFIER
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix,zero_one_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,zero_one_loss
from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from time import process_time
bgclassifier = BaggingClassifier(base_estimator=None, n_estimators=30, bootstrap_features=False, oob_score=False, n_jobs=-1)

bgclassifier.fit(X_train_tf, train_y_clf) #faccio il train con le recensioni + l'evaluation
y_pred = bgclassifier.predict(X_test_tf) #faccio la predizione delle recensioni test 

np.set_printoptions(precision=2)
class_names = ['Bad', 'Neutral','Good']

matrix = metrics.confusion_matrix(test_y_clf, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(matrix,
                cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9),
                xticklabels=["Bad","Neutral","Good"],
                yticklabels=["Bad","Neutral","Good"],
                annot=True,
                fmt='d')
plt.title("Confusion matrix, without normalization -- MULTINOMIAL DB")
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

results = confusion_matrix(test_y_clf, y_pred)
error = zero_one_loss(test_y_clf, y_pred)
lerror = round(error,2)

FP = results.sum(axis=0) - numpy.diag(results)
FN = results.sum(axis=1) - numpy.diag(results)
TP = numpy.diag(results)
TN = results.sum() - (FP + FN + TP)
prec = TP / TP + FP
sens = TP / prec
lsens = numpy.round(sens,2)

TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP)
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)

print('\n Time Processing: \n', process_time())
print('\n Zero-one classification loss: \n', lerror)
print('\n True Positive: \n', TP)
print('\n True Negative: \n', TN)
print('\n False Positive: \n', FP)
print('\n False Negative: \n', FN)
print('\n Sensitivity: \n', lsens)
print('\n True Postive Rate: \n', TPR)
print('\n True Negative Rate: \n', TNR)
print('\n False Postive Rate: \n', FPR)
print('\n False Negative Rate: \n', FNR)

# Specificity or true negative rate
TNR = TN/(TN+FP)

print('\n The Classification report:\n', classification_report(test_y_clf, y_pred, digits=2))
accuracy = accuracy_score(test_y_clf, y_pred)
laccuracy = numpy.round(accuracy,2)
print('Accuracy:', laccuracy)

In [ ]:
def graphHotel(textName,hotelNumber):
    filtered_sent=[]
    #SELECT HOTEL TO SHOW (Galleria Park Hotel)

    df = pd.read_csv("test.csv")

    nameHotel=textName
    #creo un dataframe con le informazioni
    dfHotel = df.loc[df['name'] == nameHotel]
    dfapp=dfHotel[["reviews.text","name" ,"reviews.date","Evaluation"]]
    dfapp["class"] = ""
    dfapp["valueC"] = ""
    dfapp["valueR"] = ""

    #processing delle review
    for index, row in dfapp.iterrows():
        review=row["reviews.text"]
        test2=[]
        #cleaning
        review = review.lower()
        review = review.translate(str.maketrans('', '', string.punctuation)) #remove puntuaction
        review = re.sub('@[^\s]+','',review) #remove mention
        review = re.sub('http[^\s]+','',review) #remove link
        review=emoji_pattern.sub(r'', review) # remove emoji

        #processing
        review=word_tokenize(review) #tokenization
        for w in review:             #stop word filtering
            if w not in stop_words:
                filtered_sent.append(w)
        review=filtered_sent
        filtered_sent=[]
        for w in review:            #stemming
            filtered_sent.append(lemmatizer.lemmatize(w))
        review=filtered_sent
        filtered_sent=[]
        review = ' '.join(review)
        test2.append(review)
        X_test_tf = tf_idf.transform(test2)
        #do the prediction
        MNB = bgclassifier.predict(X_test_tf)[0]
        row["class"]=MNB

    #convert format of the date
    import datetime
    new_format = "%Y-%m-%d"
    for index, row in dfapp.iterrows():
        d1 = datetime.datetime.strptime(row["reviews.date"],"%Y-%m-%dT%H:%M:%S.%fZ")
        row["reviews.date"]=d1.strftime(new_format)

    for index, row in dfapp.iterrows():
        if(row["class"]=="Good"):
            row["valueC"]=1
        if(row["class"]=="Neutral"):
            row["valueC"]=0
        if(row["class"]=="Bad"):
            row["valueC"]=-1

    for index, row in dfapp.iterrows():
        if(row["Evaluation"]=="Good"):
            row["valueR"]=1
        if(row["Evaluation"]=="Neutral"):
            row["valueR"]=0
        if(row["Evaluation"]=="Bad"):
            row["valueR"]=-1 

    YearwiseR=dfapp.groupby('reviews.date', as_index=False)['valueR'].sum()
    YearwiseC =dfapp.groupby('reviews.date', as_index=False)['valueC'].sum()

    ax = YearwiseR.plot(linewidth=5, label="Review retrieved", color="blue")
    YearwiseC.plot (linewidth=3, linestyle='dashed', ax=ax, label="Review classified", color="orange")
    plt.legend(loc="upper left")
    plt.title("Combined Statistics on Reviews of " + nameHotel)
    plt.xlabel("# of Reviews")
    plt.ylabel("Classified as")
    plt.savefig('Hotel1.png')
    plt.show()

    #plot dell'andamento
    Yearwise=dfapp.groupby(by=('reviews.date')).sum()['valueC']
    plt.figure(figsize=(8,6))
    Yearwise.plot(linewidth=5, label="Review retrieved", color="red" , marker="+" , mec="blue" )
    plt.legend(loc="lower right")
    plt.xlabel("Date of the review")
    plt.ylabel("Sum of vote")
    namePng='Hotel'+hotelNumber+'.png'
    plt.savefig(namePng)
    plt.show()



In [ ]:
#SELECT HOTEL TO SHOW (Galleria Park Hotel)
nameHotel="The Whitney Hotel"
graphHotel(nameHotel,'1')

In [ ]:
#SELECT HOTEL TO SHOW (Hotel rex)
nameHotel="Hotel Rex San Francisco"
graphHotel(nameHotel,'2')

In [ ]:
#SELECT HOTEL TO SHOW (Orlando)
nameHotel="Orlando Continental Plaza Hotel"
graphHotel(nameHotel,'3')

## **Second Question:**
The second research focuses on the following question:

**“Which are the main topics of the sentiment analysis application in the management field?”**

A company must always keep in touch and grow along with the evolution of technology. We focused our attention in the detection of which are the main uses and possibilities of the application of the sentiment analysis through the analysis,conducted by using Natural Language Processing techniques, of papers taken from MIT Sloan school related to the management and business field.


In [ ]:
#Topic Finder for all documents
#Imports
import os
import re
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix
import pandas as pd


#ITERATION OVER FILES IN A DIRECTORY
keyword_list = ['data mining', 'text mining', 'text analysis','sentiment analysis']
# import required module
import os
# assign directory
directory = 'C:\\Users\\Rauro\\OneDrive\\Desktop\\Uni\\Business and project management\\DataCSV\\'
# iterate over files in
# that directory
list_text=[]
c=0
t=0
st=""
for filename in os.listdir(directory):
    t+=1
    print(t)
    print(filename)
    f = os.path.join(directory, filename)
    # checking if it is a file
    file=pd.read_csv(f,delimiter=',')
    for i in file.index:
        print("checking file")
        if (any(word in file["text"][i] for word in keyword_list )):
    #if(file["text"][i],contains(any(keyword_list))):
            c=c+1
            print("yes")
            st+=file["text"][i]
print("checking words")
print("files found:")
print(c)
if(c==0):
    print("No file found")
    quit()

result={'text':[0]}

df = pd.DataFrame(data=result)
df['text']=st
print(df['text'])



df["word_count"]=df['text'].apply(lambda x: len(str(x).split(" ")))
l=df["word_count"][0]

print(df[['text','word_count']].head())


df.word_count.describe()


#Identify the uncommon words

freq1= pd.Series(' '.join(df['text']).split()).value_counts()[-20:]
freq1


#Text Pre-Processing

#Removing stopwords
stop_words= set(stopwords.words("english"))
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown"]
stop_words = stop_words.union(new_words)



#Remove Punctuations
corpus=[]
for i in range(0, 1):
    print(i,"%")
    #Remove punctuations
    text= re.sub('[^a-zA-Z]', ' ', df['text'][0])
    #Convert to lower Case
    text=text.lower()
    #Remove tags
    text=re.sub('""',"",text)
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    #Remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    #Removing links
    text=re.sub('https:\/\/.*|http:\/\/.*|http.*',' ',text)
    #Convert to list from string
    text = text.split()
    #Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem= WordNetLemmatizer()
    text= [lem.lemmatize(word) for word in text if not word in stop_words]
    text= " ".join(text)
    corpus.append(text)
print(len(corpus))
l=(len(corpus[0]))



#Word cloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stop_words,
                          max_words=100,
                          max_font_size=50, 
                          random_state=42
                         ).generate(str(corpus[0]))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=900)



cv= CountVectorizer(stop_words=stop_words,ngram_range=(1,3))
X=cv.fit_transform(corpus)
list(cv.vocabulary_.keys())[:10]



#Most frequently occuring words
def get_top_n_words(corpus,n=None):
    vec= CountVectorizer().fit(corpus)
    bag_of_words=vec.transform(corpus)
    sum_words= bag_of_words.sum(axis=0)
    words_freq=[(word,sum_words[0,idx]) for word, idx in vec.vocabulary_.items()]
    words_freq= sorted(words_freq,key=lambda x: x[1],reverse=True)
    return words_freq[:n]



#Convert most freq words to dataframe for plotting bar plot
top_words=get_top_n_words(corpus,n=20)
top_df= pd.DataFrame(top_words)
top_df.columns=["Word","Frequency"]



#Barplot of most freq words
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
g = sns.barplot(x="Word", y="Frequency", data=top_df)
g.set_xticklabels(g.get_xticklabels(), rotation=30)



#Most frequently occuring Bi-grams
def get_top_n2_words(corpus,n=None):
    vec1= CountVectorizer(ngram_range=(2,2), max_features=2000).fit(corpus)
    bag_of_words=vec1.transform(corpus)
    sum_words= bag_of_words.sum(axis=0)
    words_freq=[(word,sum_words[0,idx])for word,idx in vec1.vocabulary_.items()]
    words_freq=sorted(words_freq,key=lambda x:x[1], reverse=True)
    return words_freq[:n]



#Bigram Plot
top2_words=get_top_n2_words(corpus,n=20)
top2_df=pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram","Freq"]
print(top2_df)

sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Bi-gram", y="Freq", data=top2_df)
h.set_xticklabels(h.get_xticklabels(), rotation=45)



#Tri-Grams
def get_top_n3_words(corpus,n=None):
    vec1=CountVectorizer(ngram_range=(3,3),max_features=2000).fit(corpus)
    bag_of_words=vec1.transform(corpus)
    sum_words= bag_of_words.sum(axis=0)
    words_freq=[(word,sum_words[0,idx]) for word, idx in vec1.vocabulary_.items()]
    words_freq=sorted(words_freq,key=lambda x:x[1],reverse=True)
    return words_freq[:n]



top3_words= get_top_n3_words(corpus, n=20)
top3_df=pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram","Freq"]
print(top3_df)

sns.set(rc={'figure.figsize':(13,8)})
j=sns.barplot(x="Tri-gram",y="Freq",data=top3_df)
j.set_xticklabels(j.get_xticklabels(),rotation=45)



#Implementation of TF-IDF
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)
#get feature names
feature_names=cv.get_feature_names_out()



#Ordino i risultati di tf-idf in base al punteggio
def sort_coo(coo_matrix):
    tuples= zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples,key=lambda x: (x[1],x[0]), reverse=True)



def extract_topn_from_vector(feature_names,sorted_items,topn=10):
    "get the feature names and tf-idf score of top n items"
    sorted_items= sorted_items[:topn]
    score_vals=[]
    feature_vals=[]
    #Index name and results
    for idx,score in sorted_items:
        score_vals.append(round(score,3))
        feature_vals.append(feature_names[idx])
        results={}
        for idx in range(len(feature_vals)):
            results[feature_vals[idx]]=score_vals[idx]
    return results




    #genera tf-idf per il documento
    tf_idf_vector=tfidf_transformer.transform(cv.transform(corpus))
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #top n=10 items da restituire
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    print("\nKeywords:")
    for k in keywords:
        print(k,keywords[k])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


FileNotFoundError: ignored

In [ ]:
#Topic Finder for each single document

#Imports
import os
import re
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix
import pandas as pd



#ITERATION OVER FILES IN A DIRECTORY
keyword_list = ['data mining', 'text mining', 'text analysis','sentiment analysis']
# import required module
import os
# assign directory
directory = 'C:\\Users\\Rauro\\OneDrive\\Desktop\\Uni\\Business and project management\\DataCSV\\'
# iterate over files in
# that directory
list_text=[]
c=0
t=0
st=""
for filename in os.listdir(directory):
    t+=1
    print(t)
    print(filename)
    f = os.path.join(directory, filename)
    # checking if it is a file
    file=pd.read_csv(f,delimiter=',')
    for i in file.index:
        print("checking file")
        if (any(word in file["text"][i] for word in keyword_list )):
            try:
    #If the file contains the requested words then it creates a new folder in which will be putted all the npl results
              #  dir=os.path.join("C:/Users/Rauro/OneDrive/Desktop/Uni/Business and project management/DataCSV/Results/" ,str(file["year"])) 
                dir='C:\\Users\\Rauro\\OneDrive\\Desktop\\Uni\\Business and project management\\Results\\'
                dir=dir+str(file['year'][i])
                print("DIRECTORY NAME: ")
                print(dir)
                print(str(file['year']))
                if not os.path.exists(dir):
                    os.mkdir(dir)
                dir=dir+'\\'
                c=c+1
                print("yes")
                st+=file["text"][i]
                result={'text':[0]}
                df = pd.DataFrame(data=result)
                df['text']=st
                print(file["year"][i]+" "+file["title"][i])
                #FROM HERE I WILL BE PASTING THE WHOLE PROCESS IN ORDER TO DO IT INSIDE A SINGLE IF
                print("FILE CREATION")
                print("Actual dir: "+dir+"\n")
                
                #CREAZIONE FILE TXT
                save_path = dir
                name_of_file = (file["title"][i])
                completeName = os.path.join(save_path, name_of_file+".txt")         
                fileresults = open(completeName, "a+")
                
                name_of_file2 = (file["title"][i]+"text")
                completeName = os.path.join(save_path, name_of_file2+".txt")         
                fileresults2 = open(completeName, "a+")
                print(completeName)
                
                print("STARTING WORD COUNT")
                
             
                #Text Pre-Processing
                print("STARTING PRE PROCESSING")
                
                #Removing stopwords
                stop_words= set(stopwords.words("english"))
                new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown"]
                stop_words = stop_words.union(new_words)
                #Remove Punctuations
                corpus=[]
                for i in range(0, 1):
                    print(i,"%")
                    #Remove punctuations
                    text= re.sub('[^a-zA-Z]', ' ', df['text'][0])
                    #Convert to lower Case
                    text=text.lower()
                    #Remove tags
                    text=re.sub('""',"",text)
                    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
                    #Remove special characters and digits
                    text=re.sub("(\\d|\\W)+"," ",text)
                    #Removing links
                    text=re.sub('https:\/\/.*|http:\/\/.*|http.*',' ',text)
                    #Convert to list from string
                    text = text.split()
                    #Stemming
                    ps=PorterStemmer()
                    #Lemmatisation
                    lem= WordNetLemmatizer()
                    text= [lem.lemmatize(word) for word in text if not word in stop_words]
                    text= " ".join(text)
                    corpus.append(text)
                    #fileresults.write("PRE PROCESSED TEXT \n\n")
                    #fileresults.write(text)
                    #fileresults.write('\n\n\n')
                print(len(corpus))
                l=(len(corpus[0]))
                fileresults2.write(corpus[0])

                print("WORD CLOUD")
                #Word cloud
                from os import path
                from PIL import Image
                from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
                import matplotlib.pyplot as plt
                get_ipython().run_line_magic('matplotlib', 'inline')
                wordcloud = WordCloud(
                                          background_color='white',
                                          stopwords=stop_words,
                                          max_words=100,
                                          max_font_size=50, 
                                          random_state=42
                                         ).generate(str(corpus[0]))
                print(wordcloud)
                fig = plt.figure(1)
                plt.imshow(wordcloud)
                plt.axis('off')
                plt.show()
                fig.savefig(dir+"word1.png", dpi=900)
                
                print("WCOUNT VECT")
                #Count Vectorizer
                cv= CountVectorizer(stop_words=stop_words,ngram_range=(1,3))
                X=cv.fit_transform(corpus)
                list(cv.vocabulary_.keys())[:10]
                
                #Most frequently occuring words
                def get_top_n_words(corpus,n=None):
                    vec= CountVectorizer().fit(corpus)
                    bag_of_words=vec.transform(corpus)
                    sum_words= bag_of_words.sum(axis=0)
                    words_freq=[(word,sum_words[0,idx]) for word, idx in vec.vocabulary_.items()]
                    words_freq= sorted(words_freq,key=lambda x: x[1],reverse=True)
                    return words_freq[:n]
                print("STARTING UNI GRAMS")
               
                #Convert most freq words to dataframe for plotting bar plot
                top_words=get_top_n_words(corpus,n=20)
                top_df= pd.DataFrame(top_words)
                top_df.columns=["Word","Frequency"]
                #Barplot of most freq words
                import seaborn as sns
                sns.set(rc={'figure.figsize':(15,12)})
                g = sns.barplot(x="Word", y="Frequency", data=top_df)
                g.set_xticklabels(g.get_xticklabels(), rotation=30)

                g.figure.savefig(dir+"unigram.png",dpi=1000)
                
                print("STARTING bi GRAMS")

                #Most frequently occuring Bi-grams
                def get_top_n2_words(corpus,n=None):
                    vec1= CountVectorizer(ngram_range=(2,2), max_features=2000).fit(corpus)
                    bag_of_words=vec1.transform(corpus)
                    sum_words= bag_of_words.sum(axis=0)
                    words_freq=[(word,sum_words[0,idx])for word,idx in vec1.vocabulary_.items()]
                    words_freq=sorted(words_freq,key=lambda x:x[1], reverse=True)
                    return words_freq[:n]
                #Bigram Plot
                top2_words=get_top_n2_words(corpus,n=20)
                top2_df=pd.DataFrame(top2_words)
                top2_df.columns=["Bi-gram","Freq"]
                print(top2_df)

                sns.set(rc={'figure.figsize':(15,12)})
                h=sns.barplot(x="Bi-gram", y="Freq", data=top2_df)
                h.set_xticklabels(h.get_xticklabels(), rotation=45)
                h.figure.savefig(dir+"bigram.png",dpi=1000)
                
                print("STARTING TRI GRAMS")
                #Tri-Grams
                def get_top_n3_words(corpus,n=None):
                    vec1=CountVectorizer(ngram_range=(3,3),max_features=2000).fit(corpus)
                    bag_of_words=vec1.transform(corpus)
                    sum_words= bag_of_words.sum(axis=0)
                    words_freq=[(word,sum_words[0,idx]) for word, idx in vec1.vocabulary_.items()]
                    words_freq=sorted(words_freq,key=lambda x:x[1],reverse=True)
                    return words_freq[:n]
                top3_words= get_top_n3_words(corpus, n=20)

                top3_df=pd.DataFrame(top3_words)
                top3_df.columns=["Tri-gram","Freq"]
                print(top3_df)

                sns.set(rc={'figure.figsize':(15,12)})
                j=sns.barplot(x="Tri-gram",y="Freq",data=top3_df)
                j.set_xticklabels(j.get_xticklabels(),rotation=45)
                j.figure.savefig(dir+"trigram.png",dpi=1000)
                
                print("STARTING TI IDF")
                #Implementation of TF-IDF
                tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
                tfidf_transformer.fit(X)
                #get feature names
                feature_names=cv.get_feature_names_out()
                
                #Ordino i risultati di tf-idf in base al punteggio
                def sort_coo(coo_matrix):
                    tuples= zip(coo_matrix.col, coo_matrix.data)
                    return sorted(tuples,key=lambda x: (x[1],x[0]), reverse=True)
                
                def extract_topn_from_vector(feature_names,sorted_items,topn=10):
                    "get the feature names and tf-idf score of top n items"
                    sorted_items= sorted_items[:topn]
                    score_vals=[]
                    feature_vals=[]
                    #Index name and results
                    for idx,score in sorted_items:
                        score_vals.append(round(score,3))
                        feature_vals.append(feature_names[idx])
                        results={}
                        for idx in range(len(feature_vals)):
                            results[feature_vals[idx]]=score_vals[idx]
                    return results
            
            
                #genera tf-idf per il documento
                tf_idf_vector=tfidf_transformer.transform(cv.transform(corpus))
                sorted_items=sort_coo(tf_idf_vector.tocoo())

                #top n=10 items da restituire
                keywords=extract_topn_from_vector(feature_names,sorted_items,10)

                print("\nKeywords:")
                fileresults.write("Keywords\n")
                for k in keywords:
                    print(k,keywords[k])
                    fileresults.write(k+" "+ str(keywords[k])+"\n")
                fileresults.close()
                
            
            
            #FROM HERE WE END
            except:
                print("Cartella non creata")       
print("checking words")
print("files found:")
print(c)
if(c==0):
    print("No file found")
    quit()
for i in files:
    print(i["title"])


        



